## Exploratory Data Analysis

User Activity

In [3]:
import pandas as pd

# CSV-Datei laden
df = pd.read_csv('../data/mlog_stats.csv')

# Erste 5 Zeilen des Datensatzes anzeigen
print(df.head())


           mlogId  dt  userImprssionCount  userClickCount  userLikeCount  \
0  KCNCLCICKCHCLC  26                7907              71              1   
1  KCOCNCHCGCHCIC  30                   1               0              0   
2  NCKCMCKCNCNCOC  11                 169              15              1   
3  NCJCOCJCLCGCJC  14                 324              27              2   
4  NCGCOCKCOCOCKC  17                 154              10              2   

   userCommentCount  userShareCount  userViewCommentCount  \
0                 0               0                     3   
1                 0               0                     0   
2                 0               0                     0   
3                 1               0                     7   
4                 0               0                     4   

   userIntoPersonalHomepageCount  userFollowCreatorCount  
0                              0                       0  
1                              0                       0  